# I am using the cars1-2.dta Dataset

In [1]:
# load necessary packages python 
import pyblp
import numpy as np
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'

In [2]:
# load data
data = pd.read_stata('../data/cars1-2.dta')

/opt/anaconda3/lib/python3.8/site-packages/pandas/io/stata.py:1417: UnicodeWarning: 
One or more strings in the dta file could not be decoded using utf-8, and
so the fallback encoding of latin-1 is being used.  This can happen when a file
has been incorrectly encoded by Stata or some other software. You should verify
the string values returned are correct.
  warnings.warn(msg, UnicodeWarning)


In [3]:
data.columns

Index(['year', 'country', 'co', 'model', 'segment', 'domestic', 'firm',
       'brand', 'loc', 'qu', 'pr', 'princ', 'price', 'horsepower', 'fuel',
       'width', 'height', 'weight', 'pop', 'ngdp', 'ngdpe', 'country1',
       'country2', 'country3', 'country4', 'country5', 'yearsquared'],
      dtype='object')

In [4]:
class blp_pset:
    """
    A class used to represent a BLP (Berry, Levinsohn, and Pakes) model.
    Attributes
    ----------
    data : any
        The data used for the BLP model.
    Methods
    -------
    """
    def __init__(self, data):
        self.data = data
        

    def return_data(self):
        return self.data
    
    def define_market_share(self,market_share_assumption):
        # market share assumption
        # i will just assume total market share is a multiple of sales        
        # for each loc and year
        # first find total sale by adding up the qu
        
        data_copy = self.data.copy()
        data_copy['total_sales'] = data_copy.groupby(['country', 'year'])['qu'].transform('sum')
        # then calculate market share
        data_copy['market_share'] = data_copy['qu'] / (data_copy['total_sales'] * market_share_assumption)
        # then drop total_sales
        data_copy.drop(columns=['total_sales'], inplace=True)
        # define a outside share column
        data_copy['outside_share'] = 1 - data_copy.groupby(['country', 'year'])['market_share'].transform('sum')
        return data_copy
    
    def set_up_variables(self,dataset):
        # rename columns
        # market_ids = for each loc and year
        dataset['market_ids'] = dataset['country'].astype(str) + '_' + dataset['year'].astype(str)
        # product_ids = model
        dataset['product_ids'] = dataset['model']
        # shares =  market_share
        dataset['shares'] = dataset['market_share']
        # prices --> price
        dataset['prices'] = dataset['price']
        return dataset
    
    def create_instruments(self,dataset,exo_char,instrument):
        # this function creates instruments
        if instrument == 'own':
            dataset['demand_instruments0'] = dataset['prices']
            
        elif instrument == 'hausman':
            #calulate the average price of a model in all other markets
            # hausman = (total sales - own market sales)/(total quantity - own quantity)
            # should not use hausman instrument with C(market_ids) + C(product_ids) as within a market there is no first stage
            dataset['s'] = dataset['prices']*dataset['qu']
            dataset['ts'] = dataset.groupby(['model','year'])['s'].transform('sum')
            dataset['tqu'] = dataset.groupby(['model','year'])['qu'].transform('sum')
            dataset['demand_instruments0'] = (dataset['ts'] - dataset['s'])/(dataset['tqu'] - dataset['qu'])
            # drop columns s, ts, tqu
            dataset.drop(columns=['s', 'ts', 'tqu'], inplace=True)
            # check if demand_instruments has any missing or nan or infinities
            if dataset['demand_instruments0'].isnull().any() or not np.isfinite(dataset['demand_instruments0']).all():
                print("demand_instruments0 contains missing values, NaNs or infinities.")
                # keep only rows with missing or nan or infinities in demand_instruments0
                print(f'orginal shape {dataset.shape}')
                dataset = dataset[~((dataset['demand_instruments0'].isnull()) | (~np.isfinite(dataset['demand_instruments0'])))]
                print(f'after dropping missing or nan or infinities shape {dataset.shape}')
                
            
        elif instrument == 'blp':
            for i,char in enumerate(exo_char):
                # leave out mean = (total char - own char)/(total quantity - own quantity)
                dataset[f'q_{char}'] = dataset[f'{char}']*dataset['qu']
                dataset[f'tq_{char}'] = dataset.groupby(['market_ids'])[f'q_{char}'].transform('sum')
                dataset[f'tqu'] = dataset.groupby(['market_ids'])['qu'].transform('sum')
                dataset[f'demand_instruments{i}'] = (dataset[f'tq_{char}'] - dataset[f'q_{char}'])/((dataset['tqu'] - dataset['qu']))
                # drop columns s, ts, tqu
                dataset.drop(columns=[f'q_{char}', f'tq_{char}', 'tqu'], inplace=True)
        
        elif instrument == 'nest_size':
            # find the biggest integer subscript for demand_instruments
            sub_list = [int(c.replace('demand_instruments','')) for c in dataset.columns if 'demand_instruments' in c]
            next = max(sub_list)+1
            dataset[f'demand_instruments{next}'] = dataset.groupby(['market_ids','nesting_ids'])['shares'].transform(np.size)
        
        elif instrument == 'nest_other_char':
            # find the biggest integer subscript for demand_instruments
            sub_list = [int(c.replace('demand_instruments','')) for c in dataset.columns if 'demand_instruments' in c]
            next = max(sub_list)+1
            for i,char in enumerate(exo_char):
                # leave out mean = (total char - own char)/(total quantity - own quantity)
                dataset[f'q_{char}'] = dataset[f'{char}']*dataset['qu']
                dataset[f'tq_{char}'] = dataset.groupby(['market_ids','nesting_ids'])[f'q_{char}'].transform('sum')
                dataset[f'tqu'] = dataset.groupby(['market_ids','nesting_ids'])['qu'].transform('sum')
                dataset[f'demand_instruments{next+i}'] = (dataset[f'tq_{char}'] - dataset[f'q_{char}'])/((dataset['tqu'] - dataset['qu']))
                # drop columns s, ts, tqu
                dataset.drop(columns=[f'q_{char}', f'tq_{char}', 'tqu'], inplace=True)
                # check if demand_instruments0 has any missing or nan or infinities
                if dataset[f'demand_instruments{next+i}'].isnull().any() or not np.isfinite(dataset[f'demand_instruments{next+i}']).all():
                    print(f"demand_instruments{next+i} contains missing values, NaNs or infinities.")
                    # keep only rows with missing or nan or infinities in demand_instruments0
                    print(f'orginal shape {dataset.shape}')
                    dataset = dataset[~((dataset[f'demand_instruments{next+i}'].isnull()) | (~np.isfinite(dataset[f'demand_instruments{next+i}'])))]
                    print(f'after dropping missing or nan or infinities shape {dataset.shape}')
                # check if demand_instruments has any variation - if not drop it
                if dataset[f'demand_instruments{next+i}'].nunique() == 1:
                    print(f'no variation in demand_instruments{next+i}')
                    dataset.drop(columns=[f'demand_instruments{next+i}'], inplace=True)
        
        elif instrument == 'nest_both':
            dataset = self.create_instruments(dataset,exo_char,'nest_size')
            dataset = self.create_instruments(dataset,exo_char,'nest_other_char')
        
        return dataset
        
    def run_logit(self,market_share_assumption,instrument,FE):
        # take as input market_share_assumption
        print(f'market share assumption is {market_share_assumption}')
        logit_data = self.define_market_share(market_share_assumption)
        # run logit and return results
        
        # rename columns
        logit_data = self.set_up_variables(logit_data)
        
        # define an instrument for prices
        print(f'instrument type is {instrument}')
        # define exogenous list of characteristics
        exo_char = ['domestic','horsepower','fuel','width', 'height', 'weight']
        logit_data = self.create_instruments(logit_data,exo_char,instrument)
                        
        # define an FE type
        print(f'FE type is {FE}')
        
        if FE == 'none':
            FE_list = []
        elif FE == 'product_ids':
            FE_list = ['product_ids']
        elif FE == 'market_ids':
            FE_list = ['market_ids']
        elif FE == 'product_ids+market_ids':
            FE_list = ['product_ids','market_ids']
            
        # define formulation

        reg_formulation = '+'.join(['1','prices']+exo_char)
        fe_formulation = '+'.join([f'C({d})' for d in FE_list])
        # define logit problem
        logit_problem = pyblp.Problem(pyblp.Formulation(reg_formulation, 
                                                        absorb=fe_formulation), logit_data)
        
        logit_result = logit_problem.solve(method='2s',
                                      optimization = pyblp.Optimization('trust-constr'))
        
        return logit_result,logit_data
    
    def run_logit_excercise(self):
        # run all combinations of FE, IV, Market Share choice
        # report price coef
        result_dictionary = {}
        beta_estimates = []
        #for FE in ['none','product_ids','market_ids','product_ids+market_ids']:
        for FE in ['product_ids+market_ids']:
            #for IV in ['own','hausman','blp']:
            for IV in ['blp']:
                #for market_share_assumption in [1.1,1.2,1.5]:
                for market_share_assumption in [1.1]:
                    spec_name = f'FE={FE}, IV={IV}, Market Share Assumption={market_share_assumption}'
                    logit_result = self.run_logit(market_share_assumption,IV,FE)
                    result_dictionary[spec_name] = logit_result
                    beta_dict = dict(zip(logit_result[0].beta_labels, logit_result[0].beta))
                    beta_se_dict = dict(zip(logit_result[0].beta_labels, logit_result[0].beta_se))
                    
                    beta_estimates.append((FE,IV,market_share_assumption, beta_dict['prices'][0], beta_se_dict['prices'][0]))
                    
                    
                    #compute elasticity
                    #report Belgium_1990
                    result_dictionary[spec_name] = result_dictionary[spec_name][0],result_dictionary[spec_name][1],logit_result[0].compute_elasticities(market_id='Belgium_1990')     
        
        beta_estimates = pd.DataFrame(beta_estimates,columns = ['FE','IV','Market Share Assumption','Price Beta','Beta SE'])
        
        # identify the top 10 most expensive cars in the Belgium_1990 market
        # full FE + blp iv result
        preferred_result = result_dictionary['FE=product_ids+market_ids, IV=blp, Market Share Assumption=1.1']
        # identify the top 10 most expensive cars in the Belgium_1990 market
        Belgium_1990_top10_data = preferred_result[1].sort_values('prices')[:10]
        # first create a 100 X 100 matrix where each column is -1*beta*price_i*share_i
        # reshape to (10,10)
        elasticity_matrix = -1*preferred_result[0].beta[0]*np.tile((Belgium_1990_top10_data[['shares']].values * Belgium_1990_top10_data[['prices']].values).T,(10,1))
        # replace the diagonal with the own price elasticities
        np.fill_diagonal(elasticity_matrix, preferred_result[0].beta[0] * Belgium_1990_top10_data['prices'].values * (1 - Belgium_1990_top10_data['shares'].values))
        # the i,j-th entry is the elasticity of i wrt to j
        
        return result_dictionary,beta_estimates,elasticity_matrix
    
    def run_nested_logit(self,market_share_assumption,instrument,FE,nest,nest_instrument):
        # take as input market_share_assumption
        print(f'market share assumption is {market_share_assumption}')
        nlogit_data = self.define_market_share(market_share_assumption)
        # run logit and return results
        
        # rename columns
        nlogit_data = self.set_up_variables(nlogit_data)
        
        
        # define an instrument for prices
        print(f'instrument type is {instrument}')
        # define exogenous list of characteristics
        exo_char = ['domestic','horsepower','fuel','width', 'height', 'weight']
        nlogit_data = self.create_instruments(nlogit_data,exo_char,instrument)
        
        # define nest
        nlogit_data['brand_segment'] = nlogit_data['brand'].astype(str) + '_' + nlogit_data['segment'].astype(str)
        nlogit_data['nesting_ids'] = nlogit_data[nest]
        # define nest instrument
        nlogit_data = self.create_instruments(nlogit_data,exo_char,nest_instrument)
                        
        # define an FE type
        print(f'FE type is {FE}')
        
        
        if FE == 'none':
            FE_list = []
        elif FE == 'product_ids':
            FE_list = ['product_ids']
        elif FE == 'market_ids':
            FE_list = ['market_ids']
        elif FE == 'product_ids+market_ids':
            FE_list = ['product_ids','market_ids']
            
        # define formulation

        reg_formulation = '+'.join(['1','prices']+exo_char)
        fe_formulation = '+'.join([f'C({d})' for d in FE_list])
        # define logit problem
        nlogit_problem = pyblp.Problem(pyblp.Formulation(reg_formulation, 
                                                        absorb=fe_formulation), nlogit_data)
        
        nlogit_result = nlogit_problem.solve(method='2s',optimization = pyblp.Optimization('trust-constr'),rho=0.5)
        
        return nlogit_result,nlogit_data
    
    def run_nlogit_excercise(self):
        # run all combinations of FE, IV, Market Share choice
        # report price coef
        result_dictionary = {}
        beta_estimates = []
        for FE in ['product_ids+market_ids']:
        #for FE in ['product_ids+market_ids']:
            for IV in ['blp']:
                #for IV in ['own','blp']:
                for market_share_assumption in [1.1]:
                    #for market_share_assumption in [1.1]:
                    for nest in ['brand']:
                    #for nest in ['brand','segment','brand_segment']:
                        for nest_instrument in ['nest_other_char']:
                        #for nest_instrument in ['nest_size','nest_other_char','nest_both']:
                
                            spec_name = f'FE={FE}, IV={IV}, nest={nest}, nest_instrument={nest_instrument}, Market Share Assumption={market_share_assumption}'
                            nlogit_result = self.run_nested_logit(market_share_assumption,IV,FE,nest,nest_instrument)
                            result_dictionary[spec_name] = nlogit_result
                            beta_dict = dict(zip(nlogit_result[0].beta_labels, nlogit_result[0].beta))
                            beta_se_dict = dict(zip(nlogit_result[0].beta_labels, nlogit_result[0].beta_se))
                    
                            beta_estimates.append((FE,IV,market_share_assumption,nest,nest_instrument, 
                                                   beta_dict['prices'][0], 
                                                   beta_se_dict['prices'][0],
                                                   nlogit_result[0].rho[0][0],
                                                   nlogit_result[0].rho_se[0][0]))
                    
                    
                            #compute elasticity
                            #report Belgium_1990
                            result_dictionary[spec_name] = result_dictionary[spec_name][0],result_dictionary[spec_name][1],nlogit_result[0].compute_elasticities(market_id='Belgium_1990')     
        
        beta_estimates = pd.DataFrame(beta_estimates,columns = ['FE','IV','Market Share Assumption','nest','nest_instrument','Price Beta','Beta SE','Rho','Rho SE'])
        preferred_result = result_dictionary['FE=product_ids+market_ids, IV=blp, nest=brand, nest_instrument=nest_other_char, Market Share Assumption=1.1']
        
        #preferred_result
        elasticity_matrix_dict = {}
        for market in ['Belgium_1990','Italy_1990']:
            preferred_result_data = preferred_result[1][(preferred_result[1].market_ids == market)&
                                                    ((preferred_result[1].brand == 'Renault')|(preferred_result[1].brand == 'Fiat'))][['brand','model','price','shares']]
            preferred_result_data['group_share'] = preferred_result_data.groupby('brand')['shares'].transform('sum')
            preferred_result_data['within_share'] = preferred_result_data['shares']/preferred_result_data['group_share']
            preferred_result_data_i = preferred_result_data.copy()
            preferred_result_data_i.columns = [c+'_i' for c in preferred_result_data.columns]
            preferred_result_data_j = preferred_result_data.copy()
            preferred_result_data_j.columns = [c+'_j' for c in preferred_result_data.columns]
            preferred_result_data_cross = preferred_result_data_i.join(preferred_result_data_j,how='cross')
            
            rho = preferred_result[0].rho[0][0]
            beta = preferred_result[0].beta[0][0]
            # own price elasticity
            same_brand_same_model = (preferred_result_data_cross.brand_i==preferred_result_data_cross.brand_j)&(preferred_result_data_cross.model_i==preferred_result_data_cross.model_j)
            preferred_result_data_cross.loc[same_brand_same_model, 'elasticity'] = (
                1 / (1 - rho) 
                * preferred_result_data_cross.loc[same_brand_same_model, 'shares_j'] 
                * (
                    1 - rho * preferred_result_data_cross.loc[same_brand_same_model, 'within_share_j'] 
                    - (1 - rho) * preferred_result_data_cross.loc[same_brand_same_model, 'shares_j']
                )
                * beta
                * (preferred_result_data_cross.loc[same_brand_same_model, 'price_j'] / preferred_result_data_cross.loc[same_brand_same_model, 'shares_i'])
                )
            # cross price elasticity - same nest
            same_brand_diff_model = (preferred_result_data_cross.brand_i==preferred_result_data_cross.brand_j)&(preferred_result_data_cross.model_i!=preferred_result_data_cross.model_j)
            preferred_result_data_cross.loc[same_brand_diff_model, 'elasticity'] = (
                -1 
                * preferred_result_data_cross.loc[same_brand_diff_model, 'shares_j'] 
                * preferred_result_data_cross.loc[same_brand_diff_model, 'shares_i']
                * ( 1 + rho/(1-rho))/ preferred_result_data_cross.loc[same_brand_diff_model, 'group_share_i']
                * beta
                * (preferred_result_data_cross.loc[same_brand_diff_model, 'price_j'] / preferred_result_data_cross.loc[same_brand_diff_model, 'shares_i'])
                )
            # cross price elasticity - diff nest
            diff_brand_diff_model = (preferred_result_data_cross.brand_i!=preferred_result_data_cross.brand_j)&(preferred_result_data_cross.model_i!=preferred_result_data_cross.model_j)
            preferred_result_data_cross.loc[diff_brand_diff_model, 'elasticity'] = (
                -1 
                * preferred_result_data_cross.loc[diff_brand_diff_model, 'shares_j'] 
                * preferred_result_data_cross.loc[diff_brand_diff_model, 'shares_i']
                * beta
                * (preferred_result_data_cross.loc[diff_brand_diff_model, 'price_j'] / preferred_result_data_cross.loc[diff_brand_diff_model, 'shares_i'])
                )
            # pivot to get the elasticity matrix where the i,j-th entry is the elasticity of i wrt to j
            elasticity_matrix_dict[market] = preferred_result_data_cross.pivot(index='model_i', columns='model_j', values='elasticity')
        
        return result_dictionary,beta_estimates,elasticity_matrix_dict
        


# Question 1

In [5]:
pset = blp_pset(data)

In [6]:
result = pset.run_logit_excercise()

market share assumption is 1.1
instrument type is blp
FE type is product_ids+market_ids
Initializing the problem ...
Absorbing demand-side fixed effects ...
Initialized the problem after 00:00:00.

Dimensions:
 T     N     K1    MD    ED 
---  -----  ----  ----  ----
150  11483   7     12    2  

Formulations:
     Column Indices:          0        1          2        3      4      5       6   
--------------------------  ------  --------  ----------  ----  -----  ------  ------
X1: Linear Characteristics  prices  domestic  horsepower  fuel  width  height  weight
Solving the problem ...
Updating the weighting matrix ...
Computed results after 00:00:00.

Problem Results Summary:
GMM     Objective    Clipped  Weighting Matrix
Step      Value      Shares   Condition Number
----  -------------  -------  ----------------
 1    +7.740977E+01     0      +5.013121E+07  

Estimating standard errors ...
Computed results after 00:00:00.

Problem Results Summary:
GMM     Objective    Clipped  Weig

In [7]:
result[1]

,FE,IV,Market Share Assumption,Price Beta,Beta SE
0,product_ids+market_ids,blp,1.1,-0.088823,0.026785


# I use both product and market id fixed effects. I use the average characteristics of the other goods in the same market as my instruments for price.
# My price coefficient is -8.882255E-02.
# My elasticity estimates for the 10 most expensive cars are as follows: the i,j-th entry is the elasticity of i wrt to the price of j

In [8]:
pd.DataFrame(result[2])

,0,1,2,3,4,5,6,7,8,9
0,-0.463812,0.027475,0.001359,0.000166,0.004763,0.000107,0.000981,0.000431,0.002161,0.000386
1,0.003460,-0.442357,0.001359,0.000166,0.004763,0.000107,0.000981,0.000431,0.002161,0.000386
2,0.003460,0.027475,-0.502865,0.000166,0.004763,0.000107,0.000981,0.000431,0.002161,0.000386
3,0.003460,0.027475,0.001359,-0.515222,0.004763,0.000107,0.000981,0.000431,0.002161,0.000386
4,0.003460,0.027475,0.001359,0.000166,-0.516234,0.000107,0.000981,0.000431,0.002161,0.000386
5,0.003460,0.027475,0.001359,0.000166,0.004763,-0.528338,0.000981,0.000431,0.002161,0.000386
6,0.003460,0.027475,0.001359,0.000166,0.004763,0.000107,-0.532822,0.000431,0.002161,0.000386
7,0.003460,0.027475,0.001359,0.000166,0.004763,0.000107,0.000981,-0.534075,0.002161,0.000386
8,0.003460,0.027475,0.001359,0.000166,0.004763,0.000107,0.000981,0.000431,-0.535234,0.000386
9,0.003460,0.027475,0.001359,0.000166,0.004763,0.000107,0.000981,0.000431,0.002161,-0.538576


# Question 2
# I use the brand as my nest. My instrument is therefore 1) the number of goods in the nest 2) the average characteristics of the other goods in the nest
# Note - rho -> 0 we have the IIA logit

In [9]:
nlogit_result = pset.run_nlogit_excercise()

market share assumption is 1.1
instrument type is blp
demand_instruments6 contains missing values, NaNs or infinities.
orginal shape (11483, 42)
after dropping missing or nan or infinities shape (11032, 42)
FE type is product_ids+market_ids
Initializing the problem ...
Absorbing demand-side fixed effects ...
Initialized the problem after 00:00:00.

Dimensions:
 T     N     K1    MD    ED    H 
---  -----  ----  ----  ----  ---
150  11032   7     18    2    35 

Formulations:
     Column Indices:          0        1          2        3      4      5       6   
--------------------------  ------  --------  ----------  ----  -----  ------  ------
X1: Linear Characteristics  prices  domestic  horsepower  fuel  width  height  weight
Solving the problem ...

Rho Initial Values:
 All Groups  
-------------
+5.000000E-01

Rho Lower Bounds:
 All Groups  
-------------
+0.000000E+00

Rho Upper Bounds:
 All Groups  
-------------
+9.900000E-01

Starting optimization ...

GMM   Computation  Optimi

# Preferred Specification is product_id and market_id FE, blp instrument for price,	brand as the nest, other characteristics in the nest for my instrument for within nest shares.

In [10]:
nlogit_result[1]

,FE,IV,Market Share Assumption,nest,nest_instrument,Price Beta,Beta SE,Rho,Rho SE
0,product_ids+market_ids,blp,1.1,brand,nest_other_char,-0.130205,0.025849,0.378196,0.071389


# elasticity calculation
# see https://www.nathanhmiller.org/nlnotes.pdf


The derivative of product $j$'s share, for $j \in \mathcal{J}_g$, with respect to its own mean valuation is given by
$$
\begin{equation}
\frac{\partial s_j}{\partial \delta_j} = \frac{1}{1 - \sigma} s_j \left(1 - \sigma \bar{s}_j|_g - (1 - \sigma) s_j \right).
\end{equation}
$$
The derivative of product $j$'s share with respect to product $k$'s mean valuation, for $j$ and $k$ both in group $g$, equals
$$
\begin{equation}
\frac{\partial s_j}{\partial \delta_k} = -s_k \left( s_j + \frac{\sigma}{1 - \sigma} \bar{s}_j|_g \right).
\end{equation}
$$
The above formulation is correct but more difficult to use in obtaining cross-cross second derivatives because the symmetry between $j$ and $k$ isn't transparent. More helpful for those is the equivalent expression
$$
\begin{equation}
\frac{\partial s_j}{\partial \delta_k} = -s_k s_j \left( 1 + \frac{\sigma}{1 - \sigma} \frac{1}{\bar{s}_g} \right).
\end{equation}
$$
The derivative of product $j$'s share with respect to product $k$'s mean valuation, for $j$ and $k$ in different groups, equals
$$
\begin{equation}
\frac{\partial s_j}{\partial \delta_k} = -s_j s_k.
\end{equation}
$$
# The Matrix is as follows
# We now allow different substitution elasticities between cars of the different brands.
# Italy is more elastic in general.
# The ratio of the cross brand elasticities are similar.

In [15]:
nlogit_result[-1]['Belgium_1990']

model_j,fiat 126,fiat croma,fiat panda,fiat regata,fiat tempra,fiat tipo,fiat uno,renault 19,renault 21,renault 25,renault 4,renault 5
model_i,,,,,,,,,,,,
fiat 126,-1.273817,0.290256,0.159656,0.011594,0.144033,0.872486,0.785972,0.068830,0.040275,0.030435,0.001816,0.033074
fiat croma,0.015530,-3.607203,0.159656,0.011594,0.144033,0.872486,0.785972,0.068830,0.040275,0.030435,0.001816,0.033074
fiat panda,0.015530,0.290256,-1.456956,0.011594,0.144033,0.872486,0.785972,0.068830,0.040275,0.030435,0.001816,0.033074
fiat regata,0.015530,0.290256,0.159656,-2.752889,0.144033,0.872486,0.785972,0.068830,0.040275,0.030435,0.001816,0.033074
fiat tempra,0.015530,0.290256,0.159656,0.011594,-2.977993,0.872486,0.785972,0.068830,0.040275,0.030435,0.001816,0.033074
fiat tipo,0.015530,0.290256,0.159656,0.011594,0.144033,-2.350893,0.785972,0.068830,0.040275,0.030435,0.001816,0.033074
fiat uno,0.015530,0.290256,0.159656,0.011594,0.144033,0.872486,-1.514270,0.068830,0.040275,0.030435,0.001816,0.033074
renault 19,0.000323,0.006033,0.003319,0.000241,0.002994,0.018136,0.016338,-2.156751,0.658908,0.497930,0.029712,0.541103
renault 21,0.000323,0.006033,0.003319,0.000241,0.002994,0.018136,0.016338,1.126071,-3.145536,0.497930,0.029712,0.541103


In [16]:
nlogit_result[-1]['Italy_1990']

model_j,fiat 126,fiat croma,fiat panda,fiat regata,fiat tempra,fiat tipo,fiat uno,renault 19,renault 21,renault 25,renault 4,renault 5
model_i,,,,,,,,,,,,
fiat 126,-1.209298,0.098276,0.317521,0.002755,0.209789,0.663097,0.781690,0.032585,0.014943,0.002121,0.003987,0.024324
fiat croma,0.031629,-4.056991,0.317521,0.002755,0.209789,0.663097,0.781690,0.032585,0.014943,0.002121,0.003987,0.024324
fiat panda,0.031629,0.098276,-1.307236,0.002755,0.209789,0.663097,0.781690,0.032585,0.014943,0.002121,0.003987,0.024324
fiat regata,0.031629,0.098276,0.317521,-2.782448,0.209789,0.663097,0.781690,0.032585,0.014943,0.002121,0.003987,0.024324
fiat tempra,0.031629,0.098276,0.317521,0.002755,-2.901914,0.663097,0.781690,0.032585,0.014943,0.002121,0.003987,0.024324
fiat tipo,0.031629,0.098276,0.317521,0.002755,0.209789,-2.368463,0.781690,0.032585,0.014943,0.002121,0.003987,0.024324
fiat uno,0.031629,0.098276,0.317521,0.002755,0.209789,0.663097,-1.357019,0.032585,0.014943,0.002121,0.003987,0.024324
renault 19,0.006982,0.021693,0.070089,0.000608,0.046308,0.146370,0.172548,-2.199495,0.488770,0.069364,0.130418,0.795622
renault 21,0.006982,0.021693,0.070089,0.000608,0.046308,0.146370,0.172548,1.065816,-3.385997,0.069364,0.130418,0.795622
